In [ ]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [ ]:
years = [str(year) for year in range(2017, 2024)]
dataset_names = []
datasets = {}

# Parcourir les années et importer les jeux de données correspondants
for year in years:
    # Construire le nom de fichier pour l'année en cours
    filename = year + '.csv'
    
    # Importer le jeu de données et l'assigner à la variable
    dataset = pd.read_csv(filename)
    
    # Construire le nom de variable pour l'année en cours
    varname = 'FIFA' + year
    
    # Ajouter la colonne 'Version' avec le nom de variable
    dataset['Version'] = varname
    
    # Ajouter la colonne 'ID' en concaténant le nom de variable avec 'sofifa_id'
    dataset['ID'] = varname + dataset['sofifa_id'].astype(str)
    
    # Assigner le jeu de données à la variable en utilisant exec()
    exec(f"{varname} = dataset")
    
    # Ajouter le nom de variable à la liste dataset_names
    dataset_names.append(varname)

In [ ]:
def convert_currency(value):
    """
    Convertit les valeurs de devise du format FIFA en valeurs numériques.
    """
    if 'M' in value:
        amend_value = value.replace('€', '').replace('M', '')
        amend_value = float(amend_value) * 1000000
    elif 'K' in value:
        amend_value = value.replace('€', '').replace('K', '')
        amend_value = float(amend_value) * 1000
    else:
        amend_value = value.replace('€', '')
        amend_value = float(amend_value)
    return amend_value

def apply_convert_currency(df):
    """
    Applique la conversion des valeurs de devise à plusieurs colonnes d'un DataFrame.
    """
    df['value'] = df['value'].apply(convert_currency)
    df['wage'] = df['wage'].apply(convert_currency)
    df['release_clause'] = df['release_clause'].apply(convert_currency)
    return df

def correct_weight_height(df):
    """
    Corrige le format et les unités de poids et de taille dans un DataFrame.
    Calcule également l'indice de masse corporelle (BMI) à partir du poids et de la taille.
    """
    df['weight_kg'] = df['weight_kg'].str.split('kg').str[0].astype(float)
    df['height'] = df['height'].str.split('cm').str[0].astype(float)
    df['BMI'] = df['weight_kg'] / ((df['height'] / 100) ** 2)
    return df

def correct_league_names(df):
    """
    Corrige les noms de ligue dans un DataFrame en les fusionnant avec une table de correspondance.
    """
    leagues = pd.read_csv('league_names.csv')
    leagues = leagues.rename(columns={'league': 'league_name'})
    df = pd.merge(df, leagues, on='league_name', how='left')
    df.drop(["league_name"], axis=1, inplace=True)
    del leagues
    return df

def add_team_strength(df):
    """
    Ajoute la force de l'équipe en termes de note globale moyenne dans un DataFrame.
    """
    teams_strength = df.groupby('club_name', as_index=False)['best_overall'].mean()
    teams_strength.sort_values('best_overall', inplace=True)  # Tri des valeurs par note globale
    df = pd.merge(df, teams_strength, on='club_name', how='left')
    del teams_strength
    return df

def regroup_countries(df):
    """
    Regroupe les pays par continent dans un DataFrame en les fusionnant avec une table de correspondance.
    """
    country = pd.read_csv('country_continent.csv')
    country = country.rename(columns={'Country': 'nationality'})
    df = pd.merge(df, country, on='nationality', how='left')
    del country
    return df

In [ ]:
leagues = pd.read_csv('../Data/Fifa/Correction_Datasets/league_names.csv')

In [ ]:
def process_data(df):
    """
    Traite les données d'un DataFrame en appliquant une série de transformations.
    """
    # Supprime les lignes en double basées sur la colonne 'sofifa_id'
    df = df.drop_duplicates(subset=['sofifa_id'], keep='first')

    # Supprime les colonnes 'player_url' et 'loan_date_end'
    df.drop(["player_url", "loan_date_end"], axis=1, inplace=True)

    # Applique la conversion des valeurs de devise pour les colonnes 'value', 'wage' et 'release_clause'
    apply_convert_currency(df)

    # Calcule le nombre de jours depuis la date d'adhésion ('joined')
    df['days_since'] = (datetime.today() - pd.to_datetime(df['joined'])).dt.days

    # Corrige le format et les unités de poids, taille et calcule l'indice de masse corporelle (BMI)
    df = correct_weight_height(df)

    # Corrige les noms de ligue en les fusionnant avec une table de correspondance
    df = correct_league_names(df)

    # Ajoute la force de l'équipe en termes de note globale moyenne
    df = add_team_strength(df)

    # Regroupe les pays par continent en les fusionnant avec une table de correspondance
    df = regroup_countries(df)

    return df